## 1. Configurações Iniciais e Carregamento dos Dados

In [ ]:
# 1.1. Importações e Configuração
import pandas as pd
from pathlib import Path
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt

# Caminho para os arquivos tratados (Simulando a camada "Processed" ou MinIO/Snowflake)
BASE_PROC = Path("/home/jovyan/data/processed")
arquivos_tratados = list(BASE_PROC.glob("*_tratado.csv"))

In [ ]:
# 1.2. Carregamento e Concatenação (Lendo do 'Snowflake' Simulado)
df_list = []
for arq in arquivos_tratados:
    # A coluna de índice é o 'datetime' (índice 0)
    df_temp = pd.read_csv(arq, index_col=0, parse_dates=True)
    
    # Adicionar uma coluna de 'cidade' para futura diferenciação
    if 'petrolina' in arq.name:
        df_temp['cidade'] = 'petrolina'
    else:
        df_temp['cidade'] = 'garanhuns'
        
    df_list.append(df_temp)

df_horario = pd.concat(df_list)
print(f"Total de Registros Horários Carregados: {len(df_horario)}")

# Variáveis climáticas (excluindo 'hora', 'hora_num', 'mes')
variaveis_climaticas = [
    'temp_ar', 'umidade', 'radiacao', 'vento_vel', 'precipitacao', 'pressao'
]
df_horario = df_horario.dropna(subset=variaveis_climaticas, how='all')

## 2. Tratamento de Outliers Simples

In [ ]:
# 2.1. Funções de Capping/Outlier Treatment (Exemplo: 3 Desvios Padrão)

def tratar_outliers_capping(df, colunas, n_std=3):
    df_out = df.copy()
    print(f"Tratando outliers com o método de Capping ({n_std} Desvios Padrão)...")
    
    for col in colunas:
        mean = df_out[col].mean()
        std = df_out[col].std()
        
        # Limites inferior e superior
        lower_bound = mean - n_std * std
        upper_bound = mean + n_std * std
        
        # O capping substitui valores extremos pelos limites
        outlier_mask = (df_out[col] < lower_bound) | (df_out[col] > upper_bound)
        n_outliers = outlier_mask.sum()
        
        if n_outliers > 0:
            print(f"   - Coluna '{col}': {n_outliers} outliers detectados.")
            df_out[col] = np.where(df_out[col] < lower_bound, lower_bound, df_out[col])
            df_out[col] = np.where(df_out[col] > upper_bound, upper_bound, df_out[col])

    return df_out

# Aplicação (exclua a precipitação, que é naturalmente volátil)
cols_para_outlier = [c for c in variaveis_climaticas if c != 'precipitacao']
df_horario_limpo = tratar_outliers_capping(df_horario.copy(), cols_para_outlier)

## 3. Agregação de Features

O projeto foca no agrupamento de padrões climáticos-chave para o ciclo da uva, o que requer uma agregação temporal (semanal é um bom ponto de partida).

| Variável | Agregação Sugerida |
|---|---|
| temp_ar | "Média (mean), Desvio Padrão (std)" |
| umidade | "Média (mean), Mínima (min)" |
| radiacao | Soma (sum) |
| precipitacao | Soma (sum) |

In [ ]:
# 3.1. Agregação Semanal por Cidade
df_agregado = df_horario_limpo.groupby('cidade').resample('W').agg(
    # Temperatura
    temp_ar_mean=('temp_ar', 'mean'),
    temp_ar_std=('temp_ar', 'std'),
    
    # Umidade
    umidade_mean=('umidade', 'mean'),
    umidade_min=('umidade', 'min'),

    # Radiação e Precipitação (Acumulados)
    radiacao_sum=('radiacao', 'sum'),
    precipitacao_sum=('precipitacao', 'sum'),
    
    # Pressão (Média)
    pressao_mean=('pressao', 'mean')
)

# Limpar NaNs que podem ter surgido (e.g., semanas incompletas no início/fim ou dados totalmente faltantes)
df_agregado = df_agregado.dropna()
df_agregado = df_agregado.reset_index()

print(f"Total de Registros Agregados (Semanas): {len(df_agregado)}")

# Separar os dados para o K-Means (Excluindo a coluna 'cidade' e 'datetime')
X = df_agregado.drop(columns=['cidade', 'datetime'])

## Pré-processamento e K-means

In [ ]:
# 4.1. Escalonamento dos Dados (StandardScaler)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print("Dados escalados e prontos para o K-Means.")

# 4.2. Determinação do K Ideal (Método do Cotovelo - Elbow Method)
wcss = []
k_range = range(2, 11) # Testar de 2 a 10 clusters

for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(X_scaled)
    wcss.append(kmeans.inertia_)

# 4.3. Visualização do Método do Cotovelo
plt.figure(figsize=(10, 6))
plt.plot(k_range, wcss, marker='o', linestyle='--')
plt.title('Método do Cotovelo para K-Means')
plt.xlabel('Número de Clusters (K)')
plt.ylabel('WCSS (Soma dos Quadrados Intra-Cluster)')
plt.grid(True)
plt.show()

# Baseado no gráfico, escolha o K ideal (onde a curva "dobra")
K_OTIMO = 4 # Exemplo, ajuste após rodar o código

# 4.4. Treinamento do Modelo Final
kmeans_final = KMeans(n_clusters=K_OTIMO, random_state=42, n_init=10)
df_agregado['cluster_label'] = kmeans_final.fit_predict(X_scaled)

# 4.5. Avaliação do Score de Silhueta (Métricas de Agrupamento)
silhouette_avg = silhouette_score(X_scaled, df_agregado['cluster_label'])

print("-" * 50)
print(f"✅ Agrupamento concluído com K = {K_OTIMO}")
print(f"Score de Silhueta Médio: {silhouette_avg:.4f}")
print("-" * 50)

## 5. Análise dos Clusters e Publicação

In [ ]:
# 5.1. Caracterização dos Clusters
# Calcular a média das features para cada cluster para entender o padrão climático
cluster_perfil = df_agregado.groupby('cluster_label')[X.columns].mean()

print("Média das Variáveis por Cluster (Perfil Climático):")
print(cluster_perfil.transpose())

# Opcional: Desescalar os centroides para melhor interpretação (usando a média original)
cluster_centers = scaler.inverse_transform(kmeans_final.cluster_centers_)
cluster_centers_df = pd.DataFrame(cluster_centers, columns=X.columns)
print("\nCentroides dos Clusters (Valores Reais):")
print(cluster_centers_df.transpose())

# 5.2. Preparação para Publicação/MLFlow (Próximos Passos)
# O modelo 'kmeans_final' deve ser registrado no MLFlow (port 5000)
# Os resultados do 'df_agregado' (agora com a coluna 'cluster_label') devem ser carregados para o ThingsBoard/Trendz para visualização.